# 06 – E2E Demo


Runs the full pipeline with a couple of natural queries.


In [ ]:
using semantic_kernel_app;
using Microsoft.Extensions.Logging;

// wire real services (replace dummies)
var sp = Services.Build();
var logger = LoggerFactory.Create(b=>b.AddConsole());

var clf = new RuleIntentClassifier(logger.CreateLogger<RuleIntentClassifier>());
var sf = new SlotFiller(logger.CreateLogger<SlotFiller>());
var disp = new QueryDispatcher(new InMemoryEmployeeApi(
    new Microsoft.Extensions.Caching.Memory.MemoryCache(new Microsoft.Extensions.Caching.Memory.MemoryCacheOptions()), new AppConfig()),
    logger.CreateLogger<QueryDispatcher>());
var fmt = new AnswerFormatter(new AppConfig(), logger.CreateLogger<AnswerFormatter>());

async Task Run(string q)
{
    Console.WriteLine($"\nQ: {q}");
    var (intent, ic) = clf.Classify(q);
    var (slots, sc, clar) = sf.Fill(q);
    if (clar != null){ Console.WriteLine("Clarify: " + clar); return; }

    var spec = new QuerySpec(intent, slots);
    var data = await disp.ExecuteAsync(spec);
    var (used, text) = await fmt.FormatAsync(spec, data);
    Console.WriteLine((used?"[LLM] ":"[NO LLM] ") + text);
}

await Run("What are Rick and Summer's emails?");
await Run("Show managers in engineering hired after 2020");
